In [3]:

import soundfile as sf
import csv
import time
import threading
import sounddevice as sd
import numpy as np
from pynput import keyboard
from scipy.io.wavfile import write as wav_write

### The following code is used to record the audio and the keystrokes

*It is not used in the final project, but it is kept here for reference
*The audio recording will go on for 10 seconds. The keystroke recording will be done simultaneously but needs to be stopped manually by pressing ESC (escape)

Head's up : Dont forget to grant accessibility access the editor with which you're running the code below, in particular input monitoring for keystroke recording.


In [1]:
import numpy as np
import csv
import time
import sounddevice as sd
import threading
import scipy.io.wavfile as wav
from pynput import keyboard

# Parameters for sound recording
sample_rate = 44100  # Hz
duration = 10  # seconds
audio_data = np.zeros((duration * sample_rate, 1), dtype=np.int16)

# File paths
audio_file = 'keyboard_sound.wav'
log_file = 'key_log.csv'

# Initialize the keystroke log file
with open(log_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Key', 'Action', 'Timestamp'])

# Global variable to track the start time of audio recording
start_time_audio = None
stop_recording = False  # Flag to stop both audio and keyboard listeners

# Recording function for audio
def record_audio():
    global audio_data, start_time_audio
    print("Recording audio...")

    # Set the start time of the recording
    start_time_audio = time.time()  # Capture the timestamp when recording starts

    # Record the audio and store it in audio_data
    try : 
        audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=2, dtype='int16')
        sd.wait()
        wav.write(audio_file, sample_rate, audio_data)
        print(f"Audio saved to {audio_file}")
    except:
        print("Your computer does not support stereo recording. Defaulting to mono.")
        audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
        sd.wait()
        wav.write(audio_file, sample_rate, audio_data)
        print(f"Audio saved to {audio_file}")

    print("Audio recording finished")

    # Save the recorded audio to a WAV file

# Keystroke listener function
def on_press(key):
    if start_time_audio is None:
        return  # Don't log if the audio hasn't started yet

    timestamp = time.time() - start_time_audio  # Calculate relative timestamp
    try:
        key_str = key.char  # Normal keys
    except AttributeError:
        key_str = str(key)  # Special keys like shift, ctrl, etc.

    # Log the key press with relative timestamp
    with open(log_file, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([key_str, 'Pressed', round(timestamp, 6)])  # Round for cleaner timestamp

    print(f"Key {key_str} Pressed at {timestamp:.6f} seconds")

def on_release(key):
    if start_time_audio is None:
        return  # Don't log if the audio hasn't started yet

    timestamp = time.time() - start_time_audio  # Calculate relative timestamp
    try:
        key_str = key.char
    except AttributeError:
        key_str = str(key)

    # Log the key release with relative timestamp
    with open(log_file, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([key_str, 'Released', round(timestamp, 6)])  # Round for cleaner timestamp

    print(f"Key {key_str} Released at {timestamp:.6f} seconds")

    # Stop listener if 'Esc' key is pressed
    if key == keyboard.Key.esc:
        global stop_recording
        stop_recording = True  # Set flag to stop both recordings
        return False

# Start recording audio in a separate thread
audio_thread = threading.Thread(target=record_audio)
audio_thread.start()

# Start the keyboard listener in the main thread to avoid blocking
def start_keyboard_listener():
    with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
        listener.join()

# Run the keyboard listener in a separate thread
keyboard_thread = threading.Thread(target=start_keyboard_listener)
keyboard_thread.start()

# Wait for both threads to finish, while checking for stop condition
while not stop_recording:
    time.sleep(0.1)  # Prevent high CPU usage by sleeping briefly

# Once 'Esc' is pressed, both threads will finish
audio_thread.join()
keyboard_thread.join()
print("Recording process finished.")

This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


Recording audio...
Your computer does not support stereo recording. Defaulting to mono.
Key h Pressed at 1.552957 seconds
Key h Released at 1.654829 seconds
Key f Pressed at 1.902737 seconds
Key f Released at 1.994418 seconds
Key h Pressed at 2.226380 seconds
Key h Released at 2.342960 seconds
Key f Pressed at 2.586601 seconds
Key f Released at 2.683351 seconds
Key h Pressed at 2.892256 seconds
Key h Released at 3.010252 seconds
Key f Pressed at 3.233224 seconds
Key f Released at 3.329910 seconds
Key h Pressed at 3.558386 seconds
Key h Released at 3.644606 seconds
Key u Pressed at 4.117786 seconds
Key u Released at 4.198595 seconds
Key j Pressed at 4.403640 seconds
Key j Released at 4.492613 seconds
Key u Pressed at 4.676039 seconds
Key u Released at 4.744907 seconds
Key j Pressed at 4.902137 seconds
Key j Released at 4.992853 seconds
Key u Pressed at 5.143980 seconds
Key u Released at 5.208439 seconds
Key j Pressed at 5.350864 seconds
Key j Released at 5.433881 seconds
Key u Pressed a

In [7]:
print(sd.query_devices())

> 0 MacBook Pro Microphone, Core Audio (1 in, 0 out)
< 1 MacBook Pro Speakers, Core Audio (0 in, 2 out)
  2 Microsoft Teams Audio, Core Audio (1 in, 1 out)


In [4]:
# Load the audio file
audio_path = "keyboard_sound.wav"
audio_data, sample_rate = sf.read(audio_path)

# Read keystroke timestamps from CSV
keystroke_times = []
with open("key_log.csv", "r") as audio_data_file:
    reader = csv.reader(audio_data_file)
    stack = {}

    for row in reader:
        if row[0] == "Key":
            continue
        key = row[0]
        action = row[1]
        timestamp = float(row[2])

        if action == "Pressed":
            if key in stack:
                raise Exception(f"Key {key} was pressed again before being released.")
            stack[key] = [key, timestamp]

        elif action == "Released":
            if key not in stack:
                raise Exception(f"Key {key} was released without being pressed.")
            stack[key].append(timestamp)
            keystroke_times.append(stack[key])
            del stack[key]

print(keystroke_times)


[['h', 1.552957, 1.654829], ['f', 1.902737, 1.994418], ['h', 2.22638, 2.34296], ['f', 2.586601, 2.683351], ['h', 2.892256, 3.010252], ['f', 3.233224, 3.32991], ['h', 3.558386, 3.644606], ['u', 4.117786, 4.198595], ['j', 4.40364, 4.492613], ['u', 4.676039, 4.744907], ['j', 4.902137, 4.992853], ['u', 5.14398, 5.208439], ['j', 5.350864, 5.433881], ['u', 5.57688, 5.641931], ['j', 5.798865, 5.88013], ['k', 6.582139, 6.648088], ['l', 6.827523, 6.899125], ['i', 7.075488, 7.147173], ['o', 7.28879, 7.378165], ['u', 7.908364, 7.997006], ['i', 8.213281, 8.304807], ['o', 8.508651, 8.588767], ['p', 8.800293, 8.881917], ['o', 9.063528, 9.149904], ['i', 9.332562, 9.406096], ['u', 9.618491, 9.696694], ['y', 9.890357, 9.970163], ['t', 10.165096, 10.252894], ['y', 10.492283, 10.59244], ['Key.esc', 12.428661, 12.636896]]


## The code that generates individual spectrograms for each keystroke

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
import scipy.signal as signal
import csv
import os

# Parameters
AUDIO_FILE = "keyboard_sound.wav"  # Replace with your actual audio file
KEYSTROKE_CSV = "key_log.csv"
OUTPUT_DIR = "keystroke_spectrograms/keystroke_spectrograms"
NUMPY_OUTPUT_DIR = "keystroke_spectrograms/numpy_arrays"  # New directory for NumPy arrays

## Hyperparameter to fine-tune
BUFFER_TIME = 0.2  # Extra time (seconds) before & after each keystroke

# Ensure output directories exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(NUMPY_OUTPUT_DIR, exist_ok=True)

# Load audio data
sample_rate, audio_data = wav.read(AUDIO_FILE)

# Read keystroke data from CSV
keystroke_times = []
with open(KEYSTROKE_CSV, "r") as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    stack = {}

    for row in reader:
        key, action, timestamp = row[0], row[1], float(row[2])

        if action == "Pressed":
            stack[key] = timestamp  # Store press time
        elif action == "Released" and key in stack:
            keystroke_times.append((key, stack.pop(key), timestamp))  # Store key, press, and release

# Function to create and save the spectrogram and numpy arrays
def create_spectrogram_and_numpy(audio_segment, key, idx):
    # Generate the spectrogram using scipy
    f, t, Sxx = signal.spectrogram(audio_segment, sample_rate)

    # Normalize spectrogram for neural network (optional)
    Sxx_log = 10 * np.log10(Sxx + 1e-10)  # Adding small constant to avoid log(0)

    # Plot the spectrogram
    plt.figure(figsize=(10, 4))
    plt.pcolormesh(t, f, Sxx_log, shading='auto', cmap='inferno')
    plt.colorbar(label='Power (dB)')
    plt.title(f"Keystroke '{key}' Spectrogram")
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")

    # Save the spectrogram to file as PNG
    spectrogram_path = os.path.join(OUTPUT_DIR, f"keystroke_{idx + 1}_{key}.png")
    plt.savefig(spectrogram_path)
    plt.close()
    print(f"Saved spectrogram for '{key}' at {spectrogram_path}")

    # Save the spectrogram as a NumPy array
    numpy_array_path = os.path.join(NUMPY_OUTPUT_DIR, f"keystroke_{idx + 1}_{key}.npy")
    np.save(numpy_array_path, Sxx_log)  # Save the log-scaled spectrogram as a numpy array
    print(f"Saved NumPy array for '{key}' at {numpy_array_path}")

# Process keystrokes
for idx, (key, press_time, release_time) in enumerate(keystroke_times):
    # Define start and end times (add buffer)
    start_time = max(0, press_time - BUFFER_TIME)
    end_time = min(len(audio_data) / sample_rate, release_time + BUFFER_TIME)

    # Convert times to sample indices
    start_sample = int(start_time * sample_rate)
    end_sample = int(end_time * sample_rate)

    # Extract audio segment
    keystroke_audio = audio_data[start_sample:end_sample]

    if len(keystroke_audio) == 0:
        print(f"Warning: Empty audio segment for keystroke {idx + 1}")
        continue

    # Create and save the spectrogram and numpy array for this key
    create_spectrogram_and_numpy(keystroke_audio, key, idx)

print("Processing complete. Spectrograms and NumPy arrays saved.")


Saved spectrogram for 'h' at keystroke_spectrograms/keystroke_spectrograms/keystroke_1_h.png
Saved NumPy array for 'h' at keystroke_spectrograms/numpy_arrays/keystroke_1_h.npy
Saved spectrogram for 'f' at keystroke_spectrograms/keystroke_spectrograms/keystroke_2_f.png
Saved NumPy array for 'f' at keystroke_spectrograms/numpy_arrays/keystroke_2_f.npy
Saved spectrogram for 'h' at keystroke_spectrograms/keystroke_spectrograms/keystroke_3_h.png
Saved NumPy array for 'h' at keystroke_spectrograms/numpy_arrays/keystroke_3_h.npy
Saved spectrogram for 'f' at keystroke_spectrograms/keystroke_spectrograms/keystroke_4_f.png
Saved NumPy array for 'f' at keystroke_spectrograms/numpy_arrays/keystroke_4_f.npy
Saved spectrogram for 'h' at keystroke_spectrograms/keystroke_spectrograms/keystroke_5_h.png
Saved NumPy array for 'h' at keystroke_spectrograms/numpy_arrays/keystroke_5_h.npy
Saved spectrogram for 'f' at keystroke_spectrograms/keystroke_spectrograms/keystroke_6_f.png
Saved NumPy array for 'f' a